In [1]:
from pathlib import Path
import os
import pandas as pd
import re
import numpy as np

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [3]:
from transformers import BertTokenizerFast, BertModel, AdamW, get_linear_schedule_with_warmup

In [4]:
from project_path import project_path
project_path(1)

d:\Users\Nicholas\Projects\NLP\toxic


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, precision_recall_fscore_support

In [6]:
# import from src
import src.models.models as models
from src.models.attention import Attention

In [18]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

inputs = tokenizer(["Hello, my dog is cute", 
                    "Hello, my cat is cute", 
                    "Hello, my fish is cute",
                    "Hello, my pet is cute"], return_tensors="pt")
outputs = model(**inputs, return_dict = False)

# last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
seq, pool = outputs

In [20]:
seq.shape

torch.Size([4, 8, 768])

In [37]:
chunks_emb = seq.split_with_sizes([2,1,1])

In [38]:
y = torch.stack([torch.mean(x, 0) for x in chunks_emb])

In [39]:
y.shape

torch.Size([3, 8, 768])

In [40]:
attn = Attention(768)

In [50]:
print([x.size() for x in chunks_emb])

[torch.Size([2, 8, 768]), torch.Size([1, 8, 768]), torch.Size([1, 8, 768])]


In [51]:
print([x.view(1, -1, 768).size() for x in chunks_emb])

[torch.Size([1, 16, 768]), torch.Size([1, 8, 768]), torch.Size([1, 8, 768])]


In [52]:
y = torch.stack([attn(x.view(1, -1, 768)) for x in chunks_emb])

In [55]:
y.size()

torch.Size([3, 1, 768])